<b>[Python sample code: the single model (weak learner)] Kagawa et al. Predicting elevated natriuretic peptide in chest radiography: Emerging utilization gap for artificial intelligence. European Heart Journal - Imaging Methods and Practice, qyae064, https://doi.org/10.1093/ehjimp/qyae064</b><br>
<br>
The sample codes are utilized in Google Colab or Jupyter Notebook.

version 0.0.1 July 04, 2024

1. If you utilize in Google Colab, mount Google Colab.
2. Install tensorflow_addons.

In Google Colab or Jupyter Notebook, either click the arrow on the cell or select the cell and use the appropriate shortcut to execute it (Shift + Enter).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install tensorflow_addons

2. Import libraries.

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

3. Copy chest radiograms to image_file_path.
4. save 'model_0.h5' to model_path.
ex:
model_path = '/content/drive/MyDrive/model_0.h5'
5. define files path.
ex:
image_file_path = '/content/drive/MyDrive/image_file.png'

In [3]:
target_size = 224
model_path = '/content/drive/MyDrive/model_0.h5'
image_file_path = '/content/drive/MyDrive/image_file.png'


4. Load model<br>
Please ignore TFA warning.

In [ ]:

image_file = np.zeros((1, target_size, target_size, 3))
image_file = np.array(img_to_array(load_img(image_file_path, color_mode='rgb', target_size=(target_size, target_size))))/255.0

print ('Shape of image: ', image_file.shape)

custom_objects = {"Addons>mish": tfa.activations.mish}
model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)
#print (model.summary())

5. Predict

In [ ]:
MODEL_PREPROCESS_INPUT = tf.keras.applications.vgg16.preprocess_input

img_array = np.expand_dims(image_file, axis=0)
preprocess_input = MODEL_PREPROCESS_INPUT
tmp_probs = model.predict(preprocess_input(img_array*255))
print('File: ', image_file_path)
print('Model: ', model_path)
print('The provability of BNP >= 200 pg/mL (single model):', tmp_probs[0][1])